<p> 
<hr>
<i><H5>SET UP</H5></i>
For this project we aim to use the EC2 Instance Connect End Point to connect to resources in the VPC (Virtual Private Cloud) using the AWS CLI. Once set up an IAM User will be created with programmatic access which will be configured to our computer. This will allow us to use the EC2 Instance connect endpoint to connect to resources in the private subnet and the public subnet of our VPC using AWS CLI
</p>
<br>

<H5><i>Pre-Installation & Requirements</i><H5>
Prerequisites for the project include:

<ul>
<li>Homebrew</li>
<li>iterm2</li>
<li>AWS CLI</li>
</ul>
<hr>

<ol>
<li><i>AWS - IAMS</i></li>
(Creating new Administrative Access User)
<br>
<img src="Images/UsersSelect.png" alt="Drawing" style="width: 300px;"/>

![CreateUser](Images/CreateUser.png)

Once created Attach required policies: 

![UserName](Images/UserName.png)
![AttachPolicies](Images/AttachPolicies.png)
<hr>
<li><i>Secret Access Key for IAM User</i></li>
<br>

![ClickUser](Images/ClickUser.png)
![CreateAccessKey](Images/CreateAccessKey.png)
![CLIAttribute](Images/CLIAttribute.png)
![AccessKeyShow](Images/AccessKeyShow.png)
<hr>
<li><i>Configuring the secret access Key on the AWS CLI and ECE Instance
<br>
This will be needed to utilise AWS resource in the VP
<br>
Open Terminal and enter 
<br>
<code>
> aws configure
</code>

![aws configure](Images/awsConfigure.png)
<br>
Once set up credentials can be found through the following: 
<br>
<code>
> cd
> ls -a 
</code>

<br>
Note: "-a" syntax is used to reveal hidden folders
<br>
<code>
> cd .aws
</code>

<br>
present 2 repositories "config" and "credentials", utilise <i>cat</i> command to access contents
<br>

<code>
> cat config
<br>[default]
<br>region = eu-west
</code>

<br>
Configuration has created a default where the region has been stored
</ol>

